# Experimentación con Modelos ML - Proyecto de Grado
## Framework de RL para Optimización de Datos Sintéticos y Modelos ML

**Título:** Sintetizando datos tabulares: Un Framework de Aprendizaje por Refuerzo para el Benchmark de Datos Sintéticos y su Impacto en problemas de Clasificación

**Autor:** Carlos Andres Cortez Ballen

### Objetivos de este Notebook:
- Implementar y entrenar modelos de scoring crediticio (XGBoost, CatBoost, LightGBM, etc.)
- Evaluar modelos con métricas específicas del dominio (AUC, PSI, Traffic Light)
- Comparar performance entre modelos entrenados con datos reales vs sintéticos
- Establecer baseline de performance para el segmento D
- Preparar datos para el framework de RL

### Metodología:
1. **Modelos ML**: XGBoost, CatBoost, LightGBM, HistGradientBoosting, RandomForest, LogisticRegression
2. **Métricas de Evaluación**: AUC-ROC, PSI, Traffic Light, Gini Coefficient, Population Stability
3. **Validación**: Cross-validation y evaluación en datos de prueba
4. **Comparación**: Datos reales vs datos sintéticos

### Contenido:
1. Configuración y carga de datos
2. División de datos (train/validation/test)
3. Entrenamiento de modelos con datos reales
4. Evaluación y comparación de modelos
5. Análisis de importancia de features
6. Preparación para experimentos con datos sintéticos


In [ ]:
# Importación de librerías y configuración
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import yaml
import logging
from tqdm import tqdm
import time

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Cargar configuración
with open('../configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("✅ Librerías importadas correctamente")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")
print(f"⚙️ Configuración cargada: {len(config)} secciones")


In [ ]:
# Importar módulos del proyecto
from src.data.data_loader import DataLoader
from src.data.data_preprocessor import DataPreprocessor
from src.data.data_splitter import DataSplitter
from src.models.model_factory import ModelFactory
from src.models.model_evaluator import CreditModelEvaluator

# Inicializar componentes
data_loader = DataLoader(config)
data_preprocessor = DataPreprocessor(config)
data_splitter = DataSplitter(config)
model_factory = ModelFactory(config)
credit_evaluator = CreditModelEvaluator(config)

print("✅ Módulos del proyecto importados correctamente")
print("📁 DataLoader inicializado")
print("🔧 DataPreprocessor inicializado")
print("✂️ DataSplitter inicializado")
print("🤖 ModelFactory inicializado")
print("📊 CreditModelEvaluator inicializado")


In [ ]:
# Cargar y preparar datos del segmento D
print("🔄 Cargando y preparando datos del segmento D...")

try:
    # Cargar German Credit Data como proxy del segmento D
    german_features, german_targets = data_loader.load_uci_dataset(144, "german_credit")
    segment_d_data = pd.concat([german_features, german_targets], axis=1)
    
    print(f"✅ Datos del segmento D cargados: {segment_d_data.shape}")
    print(f"   Features: {german_features.shape[1]}")
    print(f"   Target distribution: {german_targets.iloc[:, 0].value_counts().to_dict()}")
    
    # Preprocesar datos
    print("\n🔧 Preprocesando datos...")
    processed_data = data_preprocessor.preprocess_data(segment_d_data, target_col='class', fit=True)
    
    print(f"✅ Datos preprocesados: {processed_data.shape}")
    print(f"   Feature names: {len(data_preprocessor.get_feature_names())}")
    
    # Separar features y target
    target_col = 'class'
    X = processed_data.drop(columns=[target_col])
    y = processed_data[target_col]
    
    print(f"\n📊 Datos preparados:")
    print(f"   Features (X): {X.shape}")
    print(f"   Target (y): {y.shape}")
    print(f"   Target distribution: {y.value_counts().to_dict()}")
    
except Exception as e:
    print(f"❌ Error cargando datos: {e}")
    X, y = None, None


In [ ]:
# División de datos para entrenamiento y evaluación
if X is not None and y is not None:
    print("✂️ Dividiendo datos en train/validation/test...")
    
    # Dividir datos
    splits = data_splitter.split_data(X, y, stratify=True)
    
    X_train, y_train = splits['train']
    X_val, y_val = splits['validation']
    X_test, y_test = splits['test']
    
    print(f"✅ Datos divididos:")
    print(f"   Train: {X_train.shape[0]} muestras")
    print(f"   Validation: {X_val.shape[0]} muestras")
    print(f"   Test: {X_test.shape[0]} muestras")
    
    # Mostrar distribución del target en cada split
    print(f"\n📊 Distribución del target:")
    for split_name, (X_split, y_split) in splits.items():
        print(f"   {split_name}: {y_split.value_counts().to_dict()}")
    
    # Información de los splits
    split_info = data_splitter.get_data_info(splits)
    print(f"\n📋 Información de splits:")
    for split_name, info in split_info.items():
        print(f"   {split_name}: {info['n_samples']} muestras, {info['n_features']} features")
    
else:
    print("❌ No hay datos disponibles para dividir")
    splits = None
